We open the existing DuckDB file so all downstream tables are persisted in a single database and the notebook runs against a stable state

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

print("connected")

Quick sanity check to verify that the schedules relation is accessible and non empty before any transformations are attempted.

In [ ]:
print("schedules", con.execute("SELECT COUNT(*) FROM schedules").fetchone()[0])

We expand each game into two team perspective rows and establishes the one row per team per week structure with the required primary keys

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE team_game_weeks AS
WITH base AS (
  SELECT
    s.season,
    s.week,
    s.game_id,
    TRY_CAST(CAST(s.gameday AS VARCHAR) AS DATE) AS game_date,
    s.home_team,
    s.away_team,
    s.home_score,
    s.away_score
  FROM schedules s
)
SELECT
  season,
  week,
  game_id,
  game_date,
  home_team AS team,
  away_team AS opponent,
  1 AS home_flag,
  0 AS away_flag,
  home_score AS points_for,
  away_score AS points_against
FROM base

UNION ALL

SELECT
  season,
  week,
  game_id,
  game_date,
  away_team AS team,
  home_team AS opponent,
  0 AS home_flag,
  1 AS away_flag,
  away_score AS points_for,
  home_score AS points_against
FROM base
""")

print("team_game_weeks", con.execute("SELECT COUNT(*) FROM team_game_weeks").fetchone()[0])

Quick sanity check that ensures the generated primary key, season, week, team, game_id is unique and confirms there are no accidental duplicates

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT CAST(season AS VARCHAR) || '-' || CAST(week AS VARCHAR) || '-' || team || '-' || game_id) AS distinct_keys
FROM team_game_weeks
""").df()

We checkpoint the database to reduce wal growth and closes the connection cleanly

In [ ]:
con.execute("CHECKPOINT")
con.close()

print("db successfully closed")

For each team week, attach game result and stats, injury outcomes, rest and schedule variables

We derive team perspective game result features that are used as model controls and reporting variables

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

_ = con.execute("""
CREATE OR REPLACE TABLE team_game_weeks_results AS
SELECT
  season,
  week,
  team,
  opponent,
  game_id,
  game_date,0
  home_flag,
  away_flag,
  points_for,
  points_against,
  points_for - points_against AS score_diff,
  CASE WHEN points_for > points_against THEN 1 ELSE 0 END AS win_flag,
  CASE WHEN points_for = points_against THEN 1 ELSE 0 END AS tie_flag,
  CASE WHEN ABS(points_for - points_against) >= 14 THEN 1 ELSE 0 END AS blowout_flag
FROM team_game_weeks
""")

print("created team_game_weeks_results")

con.execute("SHOW TABLES").df()
con.execute("SELECT * FROM team_game_weeks LIMIT 5").df()

Quick sanity check to check blowout rates. Blowout rate is important because using the blowout rate, we can get a flag for blowouts during games. A blowout flag is necessary because score situation changes how much players are exposed to risk and workload in a game and additionally, change special teams volume

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  AVG(blowout_flag) AS blowout_rate
FROM team_game_weeks_results
""").df()

We build a rest and scheduling table by adding prior game context, then computing days rest, short week, and bye last week flags for each team week

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

con.execute("""
CREATE OR REPLACE TABLE team_game_weeks_rest AS
WITH ordered AS (
  SELECT
    *,
    LAG(game_date) OVER (PARTITION BY season, team ORDER BY week) AS prev_game_date,
    LAG(week) OVER (PARTITION BY season, team ORDER BY week) AS prev_week
  FROM team_game_weeks_results
)
SELECT
  season,
  week,
  team,
  opponent,
  game_id,
  game_date,
  home_flag,
  away_flag,
  points_for,
  points_against,
  score_diff,
  win_flag,
  tie_flag,
  blowout_flag,
  DATE_DIFF('day', prev_game_date, game_date) AS days_rest,
  CASE WHEN DATE_DIFF('day', prev_game_date, game_date) <= 4 THEN 1 ELSE 0 END AS short_week_flag,
  CASE WHEN prev_week IS NOT NULL AND week - prev_week > 1 THEN 1 ELSE 0 END AS bye_last_week_flag
FROM ordered
""")

Quick sanity check to confirm the rest day calculations look realistic across the dataset

In [ ]:
con.execute("""
SELECT
  MIN(days_rest) AS min_days_rest,
  MAX(days_rest) AS max_days_rest,
  AVG(days_rest) AS avg_days_rest
FROM team_game_weeks_rest
WHERE days_rest IS NOT NULL
""").df()

We create a per team week stats table from play by play. Using this, we can count offensive and defensive scrimmage snaps faced by a specific team as well as compute that specific team's offensive yards per play

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

con.execute("""
CREATE OR REPLACE TABLE team_week_basic_stats AS
WITH scrimmage AS (
  SELECT
    season,
    week,
    posteam AS team,
    yards_gained
  FROM pbp
  WHERE posteam IS NOT NULL
    AND play_type IN ('run','pass','sack','qb_kneel','qb_spike')
),
def_scrimmage AS (
  SELECT
    season,
    week,
    defteam AS team
  FROM pbp
  WHERE defteam IS NOT NULL
    AND play_type IN ('run','pass','sack','qb_kneel','qb_spike')
)
SELECT
  s.season,
  s.week,
  s.team,
  COUNT(*) AS offensive_snaps,
  SUM(COALESCE(s.yards_gained, 0)) * 1.0 / NULLIF(COUNT(*), 0) AS off_yards_per_play,
  (
    SELECT COUNT(*)
    FROM def_scrimmage d
    WHERE d.season = s.season AND d.week = s.week AND d.team = s.team
  ) AS defensive_snaps
FROM scrimmage s
GROUP BY 1,2,3
""")

Quick sanity check on the created stats table to confirm that the row count exists and the yards per play range is within a reasonable numeric range for testing

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  MIN(off_yards_per_play) AS min_ypp,
  MAX(off_yards_per_play) AS max_ypp
FROM team_week_basic_stats
""").df()

We build a roster-based side mapping table. We assign each player week to either OFF, DEF, ST, or OTHER using roster position fields.

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

con.execute("""
CREATE OR REPLACE TABLE roster_player_side AS
SELECT
  season,
  week,
  team,
  gsis_id,
  COALESCE(NULLIF(ngs_position, ''), NULLIF(position, '')) AS pos,
  CASE
    WHEN COALESCE(NULLIF(ngs_position, ''), NULLIF(position, '')) IN ('QB','RB','FB','WR','TE','C','G','T','OL') THEN 'OFF'
    WHEN COALESCE(NULLIF(ngs_position, ''), NULLIF(position, '')) IN ('DL','DE','DT','NT','LB','ILB','OLB','CB','S','FS','SS') THEN 'DEF'
    WHEN COALESCE(NULLIF(ngs_position, ''), NULLIF(position, '')) IN ('K','P','LS') THEN 'ST'
    ELSE 'OTHER'
  END AS side
FROM rosters_weekly
WHERE gsis_id IS NOT NULL
""")

Quick sanity check on the side mapping distribution by confirming that the roster rows are being classified

In [ ]:
con.execute("""
SELECT side, COUNT(*) AS n
FROM roster_player_side
GROUP BY 1
ORDER BY n DESC
""").df()

We create a unique injury presence dataset by removing duplicates across team-week-player cohorts, providing a foundation for our detection model

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE injuries_team_week_players AS
SELECT DISTINCT
  season,
  week,
  team,
  gsis_id
FROM injuries
WHERE team IS NOT NULL
  AND gsis_id IS NOT NULL
""")

We computed new injuries at the team-week level by subtracting players listed the previous week and then totaling new offensive and defensive injuries using roster side mapping

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE new_injuries_next_week AS
WITH curr AS (
  SELECT season, week, team, gsis_id
  FROM injuries_team_week_players
),
nxt AS (
  SELECT season, week AS week_next, team, gsis_id
  FROM injuries_team_week_players
),
new_only AS (
  SELECT
    nxt.season,
    nxt.week_next,
    nxt.team,
    nxt.gsis_id
  FROM nxt
  LEFT JOIN curr
    ON curr.season = nxt.season
   AND curr.week = nxt.week_next - 1
   AND curr.team = nxt.team
   AND curr.gsis_id = nxt.gsis_id
  WHERE curr.gsis_id IS NULL
)
SELECT
  new_only.season,
  new_only.week_next - 1 AS week,
  new_only.team,
  SUM(CASE WHEN r.side = 'OFF' THEN 1 ELSE 0 END) AS inj_off_next,
  SUM(CASE WHEN r.side = 'DEF' THEN 1 ELSE 0 END) AS inj_def_next
FROM new_only
LEFT JOIN roster_player_side r
  ON r.season = new_only.season
 AND r.week = new_only.week_next
 AND r.team = new_only.team
 AND r.gsis_id = new_only.gsis_id
GROUP BY 1,2,3
""")

Quick sanity checks on the new injury outcome table by confirming row coverage and checks that average counts are in a plausible range

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  AVG(inj_off_next) AS avg_off,
  AVG(inj_def_next) AS avg_def
FROM new_injuries_next_week
""").df()